In [133]:
import pandas as pd
import numpy as np
import pandas_profiling

import seaborn as sns
pd.set_option('display.max_columns', 500) 
import matplotlib.pyplot as plt
#! pip install pandas-profiling

In [134]:
import d6tstack.convert_xls
from d6tstack.convert_xls import XLSSniffer
import glob
#
from d6tstack.utils import PrintLogger
import dask.dataframe as dd


In [135]:
#TN_2019_rmd = pd.read_excel('./data/2019 County Health Rankings Tennessee Data.xls', 'Ranked Measure Data', header=None)

In [136]:
#file_2019_rmd2_dropped.columns = file_2019_rmd2_dropped.iloc[0]
#file_2019_rmd2_dropped.head()

In [137]:
#file_2019_rmd2_dropped.iloc[0]

In [138]:
#selecting out the columns based on the string of the names of the columns
#file_2019_rmd_clean = file_2019_rmd.loc[:, ~file_2019_rmd.columns.str.startswith('Unnamed')]
#file_2019_rmd_clean.head()

In [139]:
# Finding out the sheet names by importing all sheets and using sheet_names
#TN_2019 = pd.ExcelFile('./data/2019 County Health Rankings Tennessee Data.xls')
#TN_2019.sheet_names

# Using d6tstack to check the consistancy between multiple xls files and heling in extracting desired sheet from all the xls files to merge into one

## Checking all state files contains the required sheet.
11 states were choosen represnting Southern USA. All counties data ranking was downloaded so I have 11 xls files, each containing multiple sheets. Before combining all of them, I want to know if all files have the same names of the sheets and contains the desired sheet such as 'Ranked Measure Data' and 'Additional Measure Data'.

Source of Data: http://www.countyhealthrankings.org/

In [140]:
cfg_fnames = list(glob.glob('data/Measure data southern states/*.xls'))
print(len(cfg_fnames))

11


so, all of the 11 files names were added to the file name list. Now I am going to check the consistancy between files using sniffer.

In [141]:
sniffer = XLSSniffer(cfg_fnames)

print('all files have same sheet count?', sniffer.all_same_count())
print('')
print('all files have same sheet names?', sniffer.all_same_names())
print('')
print('all files contain sheet?', sniffer.all_contain_sheetname('Ranked Measure Data'))
print('')
print('all files contain sheet?', sniffer.all_contain_sheetname('Additional Measure Data'))
print('')
print('detailed dataframe')
print('')
print(sniffer.df_xls_sheets.reset_index(drop=True))

all files have same sheet count? True

all files have same sheet names? True

all files contain sheet? True

all files contain sheet? True

detailed dataframe

                                         file_name sheets_count             sheets_idx                                       sheets_names
0          Georgia 2019 County Health Rankings.xls            7  [0, 1, 2, 3, 4, 5, 6]  [Introduction, Outcomes & Factors Rankings, Ou...
1         Arkansas 2019 County Health Rankings.xls            7  [0, 1, 2, 3, 4, 5, 6]  [Introduction, Outcomes & Factors Rankings, Ou...
2   South Carolina 2019 County Health Rankings.xls            7  [0, 1, 2, 3, 4, 5, 6]  [Introduction, Outcomes & Factors Rankings, Ou...
3          Florida 2019 County Health Rankings.xls            7  [0, 1, 2, 3, 4, 5, 6]  [Introduction, Outcomes & Factors Rankings, Ou...
4            Texas 2019 County Health Rankings.xls            7  [0, 1, 2, 3, 4, 5, 6]  [Introduction, Outcomes & Factors Rankings, Ou...
5         Vi

### It looks good that all the states files contains the sheets i need for my project.

In [142]:
# We can read the file to check, which state file it is.
#pd.read_excel(cfg_fnames[0]).head()

# Extracting Ranked Measure Data sheet from all the xls files and merging

In [143]:
# Testing if i can sucessfully load desired sheet from desired file. cfg_fnames will denote which file i want to use.
test_rmd = pd.read_excel(cfg_fnames[0], sheet_name='Ranked Measure Data', header=None)
#test.head()

In [144]:
Georgia = pd.read_excel(cfg_fnames[0], sheet_name='Ranked Measure Data', header = None)
Arkansas = pd.read_excel(cfg_fnames[1], sheet_name='Ranked Measure Data', header = None)
SouthCarolina = pd.read_excel(cfg_fnames[2], sheet_name='Ranked Measure Data', header = None)
Florida = pd.read_excel(cfg_fnames[3], sheet_name='Ranked Measure Data', header = None)
Texas = pd.read_excel(cfg_fnames[4], sheet_name='Ranked Measure Data', header = None)
Virginia = pd.read_excel(cfg_fnames[5], sheet_name='Ranked Measure Data', header = None)
Mississippi = pd.read_excel(cfg_fnames[6], sheet_name='Ranked Measure Data', header = None)
NorthCaroline = pd.read_excel(cfg_fnames[7], sheet_name='Ranked Measure Data', header = None)
Louisiana = pd.read_excel(cfg_fnames[8], sheet_name='Ranked Measure Data', header = None)
Tennessee = pd.read_excel(cfg_fnames[9], sheet_name='Ranked Measure Data', header = None)
Alabama = pd.read_excel(cfg_fnames[10], sheet_name='Ranked Measure Data', header = None)

In [147]:
Ranked_Measure_Data = pd.concat([Georgia, Arkansas, SouthCarolina, Florida, Texas, Virginia, Mississippi, NorthCaroline, Louisiana, Tennessee, Alabama])
#Ranked_Measure_Data.head()

In [148]:
Ranked_Measure_Data_dropped = Ranked_Measure_Data.drop([4, 5, 6, 7, 8, 9, 11, 12, 13, 15, 16, 17, 19, 20, 21, 22, 24, 25, 15, 26, 27, 28, 29, 31, \
                     32, 33, 35, 36, 37, 39, 41, 42, 43, 45, 47, 48, 49, 53, 54, 55, 58, 60, 61, 62, \
                    63, 64, 65, 68, 69, 70, 74, 78, 82, 84, 85, 86, 87, 89, 90, 91, 92, 94, \
                    95, 96, 97, 100, 102, 104, 105, 106, 110, 112, 113, 114, 115, 116, 117, 121, 122, 123, 124, 125, 126, \
                    127, 128, 129, 130, 133, 136, 137, 138, 140, 142, 144, 145, 146, 147, 148, 149, 150, 151, 152, \
                     153, 154, 155, 156, 157, 158, 159, 160, 161
                    ], axis=1)

In [149]:
#Ranked_Measure_Data_dropped.head()

In [150]:
#Assigning index 1 as column labels. Dropping index 0 and dropping index 1 after completion of assignement of column names.
RMD_2019_1 = Ranked_Measure_Data_dropped.rename(columns=Ranked_Measure_Data_dropped.iloc[1]).drop(Ranked_Measure_Data_dropped.index[0])
RMD_2019_2 = RMD_2019_1.drop([1], axis  = 0)

In [152]:
RMD_2019_2.reset_index(inplace=True)
#RMD_2019_2.head()

In [153]:
RMD_2019_2.columns = RMD_2019_2.columns.str.replace(' ', '_')
RMD_2019_2.columns = RMD_2019_2.columns.str.replace(' ', '_')
RMD_2019_2.columns = RMD_2019_2.columns.str.replace('/', '_')
RMD_2019_2.columns = RMD_2019_2.columns.str.replace('-', '_')
RMD_2019_2.columns = RMD_2019_2.columns.str.replace('%', 'pct')
RMD_2019_2.columns = RMD_2019_2.columns.str.replace('<', 'below')
RMD_2019_2.columns = RMD_2019_2.columns.str.replace('#', 'count')
RMD_2019_2.head()

,level_0,index,FIPS,State,County,Years_of_Potential_Life_Lost_Rate,pct_Fair_Poor,Physically_Unhealthy_Days,Mentally_Unhealthy_Days,pct_LBW,pct_Smokers,pct_Obese,Food_Environment_Index,pct_Physically_Inactive,pct_With_Access,pct_Excessive_Drinking,count_Alcohol_Impaired_Driving_Deaths,count_Driving_Deaths,pct_Alcohol_Impaired,count_Chlamydia_Cases,Chlamydia_Rate,Teen_Birth_Rate,count_Uninsured,pct_Uninsured,count_Primary_Care_Physicians,PCP_Rate,PCP_Ratio,count_Dentists,Dentist_Rate,Dentist_Ratio,count_Mental_Health_Providers,MHP_Rate,MHP_Ratio,Preventable_Hosp._Rate,pct_Screened,pct_Vaccinated,Cohort_Size,Graduation_Rate,count_Some_College,pct_Some_College,count_Unemployed,Labor_Force,pct_Unemployed,pct_Children_in_Poverty,80th_Percentile_Income,20th_Percentile_Income,Income_Ratio,Annual_Average_Violent_Crimes,Violent_Crime_Rate,count_Injury_Deaths,Injury_Death_Rate,Average_Daily_PM2.5,Presence_of_violation,pct_Severe_Housing_Problems
0,0,2,13000,Georgia,NaN,7718.31,18.8002,3.80603,3.75792,9.53273,17.8864,30.3,6,24.3,75.9692,15.0517,1540,6872,22.4098,62776,614.6,28.7665,1295994,14.8851,6778,65.7396,1521:1,5316,50.9714,1962:1,13271,127.246,786:1,4851,40,43,127482,80.6,1736701,62.5606,239786,5061413,4.73753,21.5,107344,21684,4.95038,39543.5,387.535,32240,63.161,10.9,NaN,17.7057
1,1,3,13001,Georgia,Appling,10750,20.8603,4.31562,4.16764,9.86651,20.2004,36.1,7.5,27.2,27.6486,14.6861,6,25,24,99,536.5,66.9782,2635,17.6089,8,43.4122,2304:1,3,16.1978,6174:1,7,37.7949,2646:1,7080,37,30,231,82.684,1675,38.7462,512,9261,5.52856,33.5,78394,15377,5.09813,70,379.198,74,80.1013,10.6,No,10.2056
2,2,4,13003,Georgia,Atkinson,10902.1,26.7664,4.86318,4.43994,7.60296,23.5527,32.1,7.3,28.2,0,14.4235,3,8,37.5,36,428.7,78.0177,1717,24.1729,NaN,NaN,NaN,1,11.9875,8342:1,NaN,NaN,NaN,7663,35,36,108,87.037,754,35.566,187,4160,4.49519,36.5,68298,16159,4.22662,NaN,NaN,44,105.958,10.4,No,15.8845
3,3,5,13005,Georgia,Bacon,11414.5,19.1576,4.3312,4.02076,9.45674,19.9992,33,7.8,28.9,61.0941,15.7571,4,17,23.5294,59,522.2,47.5454,1742,18.625,5,43.9676,2274:1,3,26.5041,3773:1,1,8.8347,11319:1,7038,36,32,137,84.6715,1258,42.1017,231,5091,4.53742,42.7,84407,15716,5.37077,NaN,NaN,50,88.5159,10.4,No,10.1266
4,4,6,13007,Georgia,Baker,NaN,18.5125,3.90393,3.78613,13.5514,18.2196,32.5,5,27.4,5.8244,13.819,6,10,60,13,408.8,36.4742,405,16.0651,NaN,NaN,NaN,0,0,3200:0,1,31.25,3200:1,7961,55,22,20,90,356,49.5132,77,1220,6.31148,40,79571,12956,6.14163,NaN,NaN,11,68.2128,10.8,No,11.6279


### Removing the row with average of each variables in a state (uniqueness in these rows is that these rows do not have county name, NaN)

In [154]:
RMD_2019_2 = RMD_2019_2[pd.notnull(RMD_2019_2['County'])]
RMD_2019_2.head()

,level_0,index,FIPS,State,County,Years_of_Potential_Life_Lost_Rate,pct_Fair_Poor,Physically_Unhealthy_Days,Mentally_Unhealthy_Days,pct_LBW,pct_Smokers,pct_Obese,Food_Environment_Index,pct_Physically_Inactive,pct_With_Access,pct_Excessive_Drinking,count_Alcohol_Impaired_Driving_Deaths,count_Driving_Deaths,pct_Alcohol_Impaired,count_Chlamydia_Cases,Chlamydia_Rate,Teen_Birth_Rate,count_Uninsured,pct_Uninsured,count_Primary_Care_Physicians,PCP_Rate,PCP_Ratio,count_Dentists,Dentist_Rate,Dentist_Ratio,count_Mental_Health_Providers,MHP_Rate,MHP_Ratio,Preventable_Hosp._Rate,pct_Screened,pct_Vaccinated,Cohort_Size,Graduation_Rate,count_Some_College,pct_Some_College,count_Unemployed,Labor_Force,pct_Unemployed,pct_Children_in_Poverty,80th_Percentile_Income,20th_Percentile_Income,Income_Ratio,Annual_Average_Violent_Crimes,Violent_Crime_Rate,count_Injury_Deaths,Injury_Death_Rate,Average_Daily_PM2.5,Presence_of_violation,pct_Severe_Housing_Problems
1,1,3,13001,Georgia,Appling,10750,20.8603,4.31562,4.16764,9.86651,20.2004,36.1,7.5,27.2,27.6486,14.6861,6,25,24,99,536.5,66.9782,2635,17.6089,8,43.4122,2304:1,3,16.1978,6174:1,7,37.7949,2646:1,7080,37,30,231,82.684,1675,38.7462,512,9261,5.52856,33.5,78394,15377,5.09813,70,379.198,74,80.1013,10.6,No,10.2056
2,2,4,13003,Georgia,Atkinson,10902.1,26.7664,4.86318,4.43994,7.60296,23.5527,32.1,7.3,28.2,0,14.4235,3,8,37.5,36,428.7,78.0177,1717,24.1729,NaN,NaN,NaN,1,11.9875,8342:1,NaN,NaN,NaN,7663,35,36,108,87.037,754,35.566,187,4160,4.49519,36.5,68298,16159,4.22662,NaN,NaN,44,105.958,10.4,No,15.8845
3,3,5,13005,Georgia,Bacon,11414.5,19.1576,4.3312,4.02076,9.45674,19.9992,33,7.8,28.9,61.0941,15.7571,4,17,23.5294,59,522.2,47.5454,1742,18.625,5,43.9676,2274:1,3,26.5041,3773:1,1,8.8347,11319:1,7038,36,32,137,84.6715,1258,42.1017,231,5091,4.53742,42.7,84407,15716,5.37077,NaN,NaN,50,88.5159,10.4,No,10.1266
4,4,6,13007,Georgia,Baker,NaN,18.5125,3.90393,3.78613,13.5514,18.2196,32.5,5,27.4,5.8244,13.819,6,10,60,13,408.8,36.4742,405,16.0651,NaN,NaN,NaN,0,0,3200:0,1,31.25,3200:1,7961,55,22,20,90,356,49.5132,77,1220,6.31148,40,79571,12956,6.14163,NaN,NaN,11,68.2128,10.8,No,11.6279
5,5,7,13009,Georgia,Baldwin,9659.22,23.5033,4.70273,4.25073,12.6496,24.1076,34.1,5.8,23.9,63.0074,13.9761,7,36,19.4444,289,635.7,17.8093,4975,15.0348,26,57.5935,1736:1,18,40.0837,2495:1,60,133.612,748:1,5451,40,40,295,88.8136,5245,52.3767,1059,17815,5.94443,32.5,82716,13434,6.15721,316,689.414,164,72.1015,11.4,Yes,20.025


In [155]:
RMD_2019 = RMD_2019_2.drop(["index"], axis =1)

In [157]:
#RMD_2019.head()

Filling NaN with the mean of the variable. Some variable has Yes and Nos and few have ratios. We need to clean up a bit before dealing with those columns.

In [158]:
# we use .str to replace and then convert to float
#RMD_2019['PCP_Ratio'] = RMD_2019.PCP_Ratio.str.replace(':*', '').astype(float)
#RMD_2019['Dentist_Ratio'] = RMD_2019.Dentist_Ratio.str.replace(':*', '').astype(float)
#RMD_2019['MHP_Ratio'] = RMD_2019.MHP_Ratio.str.replace(':*', '').astype(float)

In [159]:
RMD_2019.fillna(RMD_2019.mean(), inplace=True)

In [160]:
#RMD_2019.info()

In [161]:
#print(RMD_2019.MHP_Ratio)

In [162]:
#RMD_2019['Presence_of_violation'].str.contains('Yes').astype(int).head()

In [163]:
RMD_2019['Presence_of_violation'].fillna('Unk', inplace=True)

In [164]:
sns.countplot(RMD_2019['Presence_of_violation'],label="Count");

In [165]:
#RMD_2019.isnull().sum()

In [166]:
df_grouped = RMD_2019.groupby(
        by=['State']
    )['pct_LBW'].mean().reset_index(name='mean_pct_LBW_grouped')
df_grouped.plot(x = 'State', y = 'mean_pct_LBW_grouped', kind  = 'barh');

# Additional Measure Data Files loading and cleaning up

In [167]:
test_amd = pd.read_excel(cfg_fnames[0], sheet_name= 'Additional Measure Data', header=None)
#test_amd.head()

In [168]:
Georgia_amd = pd.read_excel(cfg_fnames[0], sheet_name='Additional Measure Data', header = None)
Arkansas_amd = pd.read_excel(cfg_fnames[1], sheet_name='Additional Measure Data', header = None)
SouthCarolina_amd = pd.read_excel(cfg_fnames[2], sheet_name='Additional Measure Data', header = None)
Florida_amd = pd.read_excel(cfg_fnames[3], sheet_name='Additional Measure Data', header = None)
Texas_amd = pd.read_excel(cfg_fnames[4], sheet_name='Additional Measure Data', header = None)
Virginia_amd = pd.read_excel(cfg_fnames[5], sheet_name='Additional Measure Data', header = None)
Mississippi_amd = pd.read_excel(cfg_fnames[6], sheet_name='Additional Measure Data', header = None)
NorthCaroline_amd = pd.read_excel(cfg_fnames[7], sheet_name='Additional Measure Data', header = None)
Louisiana_amd = pd.read_excel(cfg_fnames[8], sheet_name='Additional Measure Data', header = None)
Tennessee_amd = pd.read_excel(cfg_fnames[9], sheet_name='Additional Measure Data', header = None)
Alabama_amd = pd.read_excel(cfg_fnames[10], sheet_name='Additional Measure Data', header = None)

In [171]:
Additional_Measure_Data = pd.concat([Georgia_amd, Arkansas_amd, SouthCarolina_amd, Florida_amd, Texas_amd, Virginia_amd, Mississippi_amd, NorthCaroline_amd, Louisiana_amd, Tennessee_amd, Alabama_amd])
#Additional_Measure_Data.head()

In [172]:
Additional_Measure_Data_dropped = Additional_Measure_Data.drop([4, 5, 6, 7, 8, 9, 11, 12, 13, 14, 15, 16, 18, 19, 20, 21, 22, 25, 26, 27, 28, 29, 31, \
                     32, 34, 35, 37, 38, 39, 41, 49, 50, 52, 53, 56, 57, 60, 61, 62, \
                    63, 64, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 83, 84, 85, 86, 87, 88, \
                                        104, 105, 106, 107, 108, 109, 110
                    ], axis=1)

In [173]:
#Assignement of column names.
Additional_Measure_Data_dropped.columns = ['FIPS', 'State', 'County','Life Expectancy','Age-Adjusted Mortality','Child Mortality Rate',
                               'Count Infant Deaths','Infant Mortality Rate', '% Frequent Physical Distress', 
                               '% Frequent Mental Distress', '% Diabetic','HIV Prevalence Rate', '% Food Insecure', 
                               '# Limited Access to healthy foods', '% Limited Access to healthy foods', 
                               '# Drug Overdose Deaths', 'Drug Overdose Mortality Rate', '# Motor Vehicle Deaths',
                               'MV Mortality Rate', '% Insufficient Sleep','# Adult Uninsured', '% Adult Uninsured',
                               '# Child Uninsured', '% Child Uninsured', 'Household Income', '# Homeowners','% Homeowners', 
                               'Population', '% < 18','% 65 and over', '# African American', '% African American',
                               '# American Indian/Alaskan Native', '% American Indian/Alaskan Native', '# Asian',
                               '% Asian', '# Native Hawaiian/Other Pacific Islander', '% Native Hawaiian/Other Pacific Islander', 
                               '# Hispanic','% Hispanic', '# Non-Hispanic White', '% Non-Hispanic White']

In [174]:
Additional_Measure_Data_dropped.columns = Additional_Measure_Data_dropped.columns.str.replace(' ', '_')
Additional_Measure_Data_dropped.columns = Additional_Measure_Data_dropped.columns.str.replace('/', '_')
Additional_Measure_Data_dropped.columns = Additional_Measure_Data_dropped.columns.str.replace('-', '_')
Additional_Measure_Data_dropped.columns = Additional_Measure_Data_dropped.columns.str.replace('%', 'pct')
Additional_Measure_Data_dropped.columns = Additional_Measure_Data_dropped.columns.str.replace('<', 'below')
Additional_Measure_Data_dropped.columns = Additional_Measure_Data_dropped.columns.str.replace('#', 'count')

In [175]:
#Dropping index 0 and dropping index 0 and 1
Additional_Measure_Data_dropped = Additional_Measure_Data_dropped.drop([0], axis  = 0)
Additional_Measure_Data_dropped = Additional_Measure_Data_dropped.drop([1], axis  = 0)
#Additional_Measure_Data_dropped.head()

In [177]:
Additional_Measure_Data_dropped.reset_index(inplace=True)
#Additional_Measure_Data_dropped.head()

In [179]:
Additional_Measure_Data_dropped = Additional_Measure_Data_dropped.drop(["index"], axis =1)
#Additional_Measure_Data_dropped.head()

KeyError: "['index'] not found in axis"

In [181]:
# Removing the state mean rows from the data using unique 'NaN' in County column
AMD_2019 = Additional_Measure_Data_dropped[pd.notnull(Additional_Measure_Data_dropped['County'])]
#AMD_2019.head()

In [182]:
AMD_2019.fillna(AMD_2019.mean(), inplace=True)

### There are county names common in other state too. So i checked and found there are many county names common. such as 'Lee' (repeated 9 times). So I crossed checked if it is not duplicated and found that it is unique in each state. 

In [183]:
#Checked any duplicated entry in dataframes
#duplicated = cancer[cancer.duplicated()]
#duplicated = AMD_2019[AMD_2019.duplicated()]
#duplicated = RMD_2019[RMD_2019.duplicated()]
#duplicated

In [184]:
#cancer.County.value_counts()
#cancer[cancer['County'].str.match('Lee')]

# Rate of new cancers and Cancer death rate data
https://gis.cdc.gov/Cancer/USCS/DataViz.html

In [185]:
cfg_fnames_deaths = list(glob.glob('data/Cancer data southern states/*deaths.csv'))
print(len(cfg_fnames_deaths))

11


In [186]:
deaths = pd.concat([(pd.read_csv(cfg_fnames_deaths[0])),
                   (pd.read_csv(cfg_fnames_deaths[1])),
                   (pd.read_csv(cfg_fnames_deaths[2])),
                   (pd.read_csv(cfg_fnames_deaths[3])),
                   (pd.read_csv(cfg_fnames_deaths[4])),
                   (pd.read_csv(cfg_fnames_deaths[5])),
                   (pd.read_csv(cfg_fnames_deaths[6])),
                   (pd.read_csv(cfg_fnames_deaths[7])),
                   (pd.read_csv(cfg_fnames_deaths[8])),
                   (pd.read_csv(cfg_fnames_deaths[9])),
                   (pd.read_csv(cfg_fnames_deaths[10]))
                  ])
deaths.shape

(1142, 8)

In [187]:
cfg_fnames_newcases = list(glob.glob('data/Cancer data southern states/*cancers.csv'))
print(len(cfg_fnames_newcases))

11


In [188]:
newcases = pd.concat([(pd.read_csv(cfg_fnames_newcases[0])),
                   (pd.read_csv(cfg_fnames_newcases[1])),
                   (pd.read_csv(cfg_fnames_newcases[2])),
                   (pd.read_csv(cfg_fnames_newcases[3])),
                   (pd.read_csv(cfg_fnames_newcases[4])),
                   (pd.read_csv(cfg_fnames_newcases[5])),
                   (pd.read_csv(cfg_fnames_newcases[6])),
                   (pd.read_csv(cfg_fnames_newcases[7])),
                   (pd.read_csv(cfg_fnames_newcases[8])),
                   (pd.read_csv(cfg_fnames_newcases[9])),
                   (pd.read_csv(cfg_fnames_newcases[10]))
                  ])
newcases.shape

(1142, 8)

In [189]:
#newcases.head()

In [190]:
#newcases.info()

In [191]:
#newcases.AgeAdjustedRate.unique()
#Data has 'Data Supressed' which is can be converetd as NaN or any number below 16. I am using 15.

In [192]:
newcases["Cases_AgeAdj_Rate"] = pd.to_numeric(newcases.AgeAdjustedRate, errors='coerce')
newcases["CaseCount"] = pd.to_numeric(newcases.CaseCount, errors='coerce')
newcases["Population"] = pd.to_numeric(newcases.Population, errors='coerce')
newcases.head()

,Area,County,CancerType,Year,Sex,AgeAdjustedRate,CaseCount,Population,Cases_AgeAdj_Rate
0,Mississippi,Issaquena County,All Types of Cancer,2011-2015,Male and Female,277.6,25.0,6932,277.6
1,Mississippi,George County,All Types of Cancer,2011-2015,Male and Female,384.2,486.0,115705,384.2
2,Mississippi,Kemper County,All Types of Cancer,2011-2015,Male and Female,398.6,261.0,50999,398.6
3,Mississippi,Smith County,All Types of Cancer,2011-2015,Male and Female,395.5,406.0,81304,395.5
4,Mississippi,Choctaw County,All Types of Cancer,2011-2015,Male and Female,387.6,224.0,41763,387.6


In [193]:
newcases.County = newcases.County.str.replace('County', '')

In [194]:
newcases = newcases.drop(['CancerType', 'Year', 'Sex',
       'Population', 'AgeAdjustedRate'], axis=1)

In [195]:
deaths["Deaths_AgeAdj_Rate"] = pd.to_numeric(deaths.AgeAdjustedRate, errors='coerce')
deaths["DeathCount"] = pd.to_numeric(deaths.DeathCount, errors='coerce')
deaths["Population"] = pd.to_numeric(deaths.Population, errors='coerce')
#deaths.head()

In [196]:
deaths.County = deaths.County.str.replace('County', '')
#deaths.head()

In [197]:
deaths = deaths.drop(['CancerType', 'Year', 'Sex', 'AgeAdjustedRate'], axis=1)
#deaths.head()

In [198]:
print('Total Number of Cancer Deaths in last 5 years (2011-2015) in 11 selected South States is:', deaths.DeathCount.sum())

Total Number of Cancer Deaths in last 5 years (2011-2015) in 11 selected South States is: 936145.0


In [199]:
print('Total Number of New cases of Cancer in last 5 years (2011-2015) in 11 selected South States is:', newcases.CaseCount.sum())

Total Number of New cases of Cancer in last 5 years (2011-2015) in 11 selected South States is: 2491360.0


In [200]:
cancer = pd.merge(newcases, deaths)
#cancer.head()

In [201]:
cancer.columns = ['State', 'County', 'CaseCount', 'Cases_AgeAdj_Rate', 'DeathCount',
       'Population', 'Deaths_AgeAdj_Rate']

In [202]:
cancer_grouped = cancer.groupby(
        by=['State']
    )['Cases_AgeAdj_Rate'].sum().reset_index(name='rate_newcases')
cancer_grouped.plot(x = 'State', y = 'rate_newcases', kind  = 'barh');

In [203]:
cancer_grouped1 = cancer.groupby(
        by=['State']
    )['Deaths_AgeAdj_Rate'].sum().reset_index(name='rate_deaths')
cancer_grouped1.plot(x = 'State', y = 'rate_deaths', kind  = 'barh');

In [204]:
cancer_grouped2 = cancer.groupby(
        by=['State']
    )['Population'].sum().reset_index(name='pop')
cancer_grouped2.plot(x = 'State', y = 'pop', kind  = 'barh');

In [205]:
cancer.Population.describe()

count    1.142000e+03
mean     4.418066e+05
std      1.190241e+06
min      4.810000e+02
25%      6.999100e+04
50%      1.353950e+05
75%      3.393495e+05
max      2.176108e+07
Name: Population, dtype: float64

In [206]:
cancer.Population.hist()

In [207]:
cancer_grouped2 = cancer.groupby(
        by=['County']
    )['Deaths_AgeAdj_Rate'].sum().reset_index(name='Total_deaths')
cancer_grouped2.plot(x = 'County', y = 'Total_deaths', bins= 100, kind  = 'hist');

# Using Pandas Profiling to explore the data quickly

In [208]:
#profile_cancer = pandas_profiling.ProfileReport(cancer)
#profile_cancer.to_file(outputfile="output.html");

In [209]:
#profile_AMD = pandas_profiling.ProfileReport(AMD_2019)
#profile_AMD.to_file(outputfile="output_AMD.html");

In [210]:
#profile_RMD = pandas_profiling.ProfileReport(RMD_2019)
#profile_RMD.to_file(outputfile="output_RMD.html");

In [211]:
merged = pd.merge(AMD_2019, RMD_2019, on = ['FIPS'], how = 'left')
merged.shape

(1142, 95)

In [212]:
merged.head()

,level_0_x,FIPS,State_x,County_x,Life_Expectancy,Age_Adjusted_Mortality,Child_Mortality_Rate,Count_Infant_Deaths,Infant_Mortality_Rate,pct_Frequent_Physical_Distress,pct_Frequent_Mental_Distress,pct_Diabetic,HIV_Prevalence_Rate,pct_Food_Insecure,count_Limited_Access_to_healthy_foods,pct_Limited_Access_to_healthy_foods,count_Drug_Overdose_Deaths,Drug_Overdose_Mortality_Rate,count_Motor_Vehicle_Deaths,MV_Mortality_Rate,pct_Insufficient_Sleep,count_Adult_Uninsured,pct_Adult_Uninsured,count_Child_Uninsured,pct_Child_Uninsured,Household_Income,count_Homeowners,pct_Homeowners,Population,pct_below_18,pct_65_and_over,count_African_American,pct_African_American,count_American_Indian_Alaskan_Native,pct_American_Indian_Alaskan_Native,count_Asian,pct_Asian,count_Native_Hawaiian_Other_Pacific_Islander,pct_Native_Hawaiian_Other_Pacific_Islander,count_Hispanic,pct_Hispanic,count_Non_Hispanic_White,pct_Non_Hispanic_White,level_0_y,State_y,County_y,Years_of_Potential_Life_Lost_Rate,pct_Fair_Poor,Physically_Unhealthy_Days,Mentally_Unhealthy_Days,pct_LBW,pct_Smokers,pct_Obese,Food_Environment_Index,pct_Physically_Inactive,pct_With_Access,pct_Excessive_Drinking,count_Alcohol_Impaired_Driving_Deaths,count_Driving_Deaths,pct_Alcohol_Impaired,count_Chlamydia_Cases,Chlamydia_Rate,Teen_Birth_Rate,count_Uninsured,pct_Uninsured,count_Primary_Care_Physicians,PCP_Rate,PCP_Ratio,count_Dentists,Dentist_Rate,Dentist_Ratio,count_Mental_Health_Providers,MHP_Rate,MHP_Ratio,Preventable_Hosp._Rate,pct_Screened,pct_Vaccinated,Cohort_Size,Graduation_Rate,count_Some_College,pct_Some_College,count_Unemployed,Labor_Force,pct_Unemployed,pct_Children_in_Poverty,80th_Percentile_Income,20th_Percentile_Income,Income_Ratio,Annual_Average_Violent_Crimes,Violent_Crime_Rate,count_Injury_Deaths,Injury_Death_Rate,Average_Daily_PM2.5,Presence_of_violation,pct_Severe_Housing_Problems
0,1,13001,Georgia,Appling,74.553535,536.4,59.436970,107.079767,7.636473,13.513213,13.631619,12.9,172.0,14.3,1235.131823,6.773041,78.223833,18.51183,30.0000,23.225027,36.040842,2281,21.788136,384,8.102975,40862,4816,71.859147,18521,24.793478,17.266886,3444,18.595108,107,0.577723,146,0.788294,38,0.205173,1900,10.258625,12789,69.051347,1,Georgia,Appling,10749.968790,20.860321,4.315623,4.167643,9.866512,20.200401,36.1,7.5,27.2,27.648607,14.686055,6.0,25.0,24.000000,99.0,536.5,66.978193,2635,17.608928,8.000000,43.412200,2304:1,3.0,16.19783,6174:1,7.000000,37.794940,2646:1,7080.0,37.0,30.0,231.0,82.683983,1675,38.746241,512,9261,5.528561,33.5,78394,15377,5.098134,70.000000,379.198267,74.0,80.101317,10.6,No,10.205580
1,2,13003,Georgia,Atkinson,73.462140,565.4,69.199726,107.079767,7.636473,15.746570,14.973510,12.8,240.8,12.8,1027.366526,12.267063,78.223833,18.51183,21.0000,36.068220,38.392034,1491,30.622304,243,10.314092,34821,2100,74.074074,8342,27.307600,14.241189,1343,16.099257,125,1.498442,76,0.911053,89,1.066890,2070,24.814193,4739,56.808919,2,Georgia,Atkinson,10902.115783,26.766415,4.863178,4.439941,7.602957,23.552669,32.1,7.3,28.2,0.000000,14.423521,3.0,8.0,37.500000,36.0,428.7,78.017664,1717,24.172885,64.174152,47.606706,NaN,1.0,11.98753,8342:1,151.018164,105.250941,NaN,7663.0,35.0,36.0,108.0,87.037037,754,35.566038,187,4160,4.495192,36.5,68298,16159,4.226623,395.440056,320.081387,44.0,105.957713,10.4,No,15.884477
2,3,13005,Georgia,Bacon,72.577063,608.3,69.199726,107.079767,7.636473,13.213764,13.127666,12.4,468.6,13.2,607.656003,5.476352,78.223833,18.51183,25.0000,31.690498,36.434163,1503,23.108856,259,8.607511,38417,2821,70.719479,11319,25.894514,15.434226,1816,16.043820,40,0.353388,68,0.600760,19,0.167859,1008,8.905380,8249,72.877463,3,Georgia,Bacon,11414.503045,19.157643,4.331201,4.020762,9.456740,19.999158,33.0,7.8,28.9,61.094088,15.757069,4.0,17.0,23.529412,59.0,522.2,47.545419,1742,18.625040,5.000000,43.967640,2274:1,3.0,26.50411,3773:1,1.000000,8.834700,11319:1,7038.0,36.0,32.0,137.0,84.671533,1258,42.101740,231,5091,4.537419,42.7,84407,15716,5.370769,395.440056,320.081387,50.0,88.515942,10.4,N

In [213]:
merged = merged.rename(index=str, columns={'State_x': 'State', "County_x": "County"})

In [214]:
merged1 = merged.drop(['Annual_Average_Violent_Crimes', 'count_Injury_Deaths', 'count_Unemployed', 
'count_Some_College', 'count_Mental_Health_Providers', 'count_Dentists', 'count_Primary_Care_Physicians',
'count_Uninsured', 'count_Chlamydia_Cases', 'count_Driving_Deaths', 'count_Alcohol_Impaired_Driving_Deaths',
'count_Homeowners', 'count_Child_Uninsured', 'count_Adult_Uninsured', 'count_Limited_Access_to_healthy_foods',
'Count_Infant_Deaths', 'Labor_Force', 'Cohort_Size', 'count_Non_Hispanic_White', 'count_Hispanic', 
'count_Native_Hawaiian_Other_Pacific_Islander', 'count_Asian', 'count_American_Indian_Alaskan_Native', 
'count_African_American', 'count_Motor_Vehicle_Deaths', 'count_Drug_Overdose_Deaths', 'State_y', 'County_y',
], axis = 1)

In [215]:
#profile_merged = pandas_profiling.ProfileReport(merged1)
#profile_merged.to_file(outputfile="merged1.html");

In [216]:
#profile_merged1 = pandas_profiling.ProfileReport(merged1.iloc[:,0:40])
#profile_merged1.to_file(outputfile="merged40.html");

In [217]:
merged1['Injury_Death_Rate'].hist(bins=100);

# Merging SDoH and Cancer data

In cancer registry, any number below 16 is suppressed as it is not cosidered as significant; therefore, I am filling it with '1'. Data merging was not straightforward. I had to cleanup the space in county columns after the name of the couties and in some counties the name had extra words. after cleaning up I merged the df on State and County columns.

In [218]:
cancer = cancer.fillna('1')
#cancer.isnull().sum()

In [219]:
cancer.County = cancer.County.str.replace(' Parish', '')

In [220]:
cancer.County = cancer.County.str.replace('Alleghany Cnty and Clifton Forge Cty', 'Alleghany')
cancer.County = cancer.County.str.replace('Bedford City and', 'Bedford')
cancer.County = cancer.County.str.replace('Halifax  with South Boston City', 'Halifax')
#print('_'+ cancer.loc[102,'County']+'_')

In [221]:
cancer['County'] = cancer['County'].str.strip()
cancer['State'] = cancer['State'].str.strip()
merged1['County'] = merged1['County'].str.strip()
merged1['State'] = merged1['State'].str.strip()

In [222]:
#cancer[cancer['County'].str.contains('Halifax')].sort_values(by = 'County').head()
#print('_'+ cancer.loc[1097,'County']+'_')

In [223]:
master_df = pd.merge(merged1, cancer, on = ['County', 'State'], how='left')

In [224]:
master_df.head()

,level_0_x,FIPS,State,County,Life_Expectancy,Age_Adjusted_Mortality,Child_Mortality_Rate,Infant_Mortality_Rate,pct_Frequent_Physical_Distress,pct_Frequent_Mental_Distress,pct_Diabetic,HIV_Prevalence_Rate,pct_Food_Insecure,pct_Limited_Access_to_healthy_foods,Drug_Overdose_Mortality_Rate,MV_Mortality_Rate,pct_Insufficient_Sleep,pct_Adult_Uninsured,pct_Child_Uninsured,Household_Income,pct_Homeowners,Population_x,pct_below_18,pct_65_and_over,pct_African_American,pct_American_Indian_Alaskan_Native,pct_Asian,pct_Native_Hawaiian_Other_Pacific_Islander,pct_Hispanic,pct_Non_Hispanic_White,level_0_y,Years_of_Potential_Life_Lost_Rate,pct_Fair_Poor,Physically_Unhealthy_Days,Mentally_Unhealthy_Days,pct_LBW,pct_Smokers,pct_Obese,Food_Environment_Index,pct_Physically_Inactive,pct_With_Access,pct_Excessive_Drinking,pct_Alcohol_Impaired,Chlamydia_Rate,Teen_Birth_Rate,pct_Uninsured,PCP_Rate,PCP_Ratio,Dentist_Rate,Dentist_Ratio,MHP_Rate,MHP_Ratio,Preventable_Hosp._Rate,pct_Screened,pct_Vaccinated,Graduation_Rate,pct_Some_College,pct_Unemployed,pct_Children_in_Poverty,80th_Percentile_Income,20th_Percentile_Income,Income_Ratio,Violent_Crime_Rate,Injury_Death_Rate,Average_Daily_PM2.5,Presence_of_violation,pct_Severe_Housing_Problems,CaseCount,Cases_AgeAdj_Rate,DeathCount,Population_y,Deaths_AgeAdj_Rate
0,1,13001,Georgia,Appling,74.553535,536.4,59.436970,7.636473,13.513213,13.631619,12.9,172.0,14.3,6.773041,18.51183,23.225027,36.040842,21.788136,8.102975,40862,71.859147,18521,24.793478,17.266886,18.595108,0.577723,0.788294,0.205173,10.258625,69.051347,1,10749.968790,20.860321,4.315623,4.167643,9.866512,20.200401,36.1,7.5,27.2,27.648607,14.686055,24.000000,536.5,66.978193,17.608928,43.412200,2304:1,16.19783,6174:1,37.794940,2646:1,7080.0,37.0,30.0,82.683983,38.746241,5.528561,33.5,78394,15377,5.098134,379.198267,80.101317,10.6,No,10.205580,443,402.5,191,92074,179.5
1,2,13003,Georgia,Atkinson,73.462140,565.4,69.199726,7.636473,15.746570,14.973510,12.8,240.8,12.8,12.267063,18.51183,36.068220,38.392034,30.622304,10.314092,34821,74.074074,8342,27.307600,14.241189,16.099257,1.498442,0.911053,1.066890,24.814193,56.808919,2,10902.115783,26.766415,4.863178,4.439941,7.602957,23.552669,32.1,7.3,28.2,0.000000,14.423521,37.500000,428.7,78.017664,24.172885,47.606706,NaN,11.98753,8342:1,105.250941,NaN,7663.0,35.0,36.0,87.037037,35.566038,4.495192,36.5,68298,16159,4.226623,320.081387,105.957713,10.4,No,15.884477,173,427.4,79,41429,203
2,3,13005,Georgia,Bacon,72.577063,608.3,69.199726,7.636473,13.213764,13.127666,12.4,468.6,13.2,5.476352,18.51183,31.690498,36.434163,23.108856,8.607511,38417,70.719479,11319,25.894514,15.434226,16.043820,0.353388,0.600760,0.167859,8.905380,72.877463,3,11414.503045,19.157643,4.331201,4.020762,9.456740,19.999158,33.0,7.8,28.9,61.094088,15.757069,23.529412,522.2,47.545419,18.625040,43.967640,2274:1,26.50411,3773:1,8.834700,11319:1,7038.0,36.0,32.0,84.671533,42.101740,4.537419,42.7,84407,15716,5.370769,320.081387,88.515942,10.4,No,10.126582,288,452.9,138,56072,215.3
3,4,13007,Georgia,Baker,76.226226,512.5,69.199726,7.636473,11.841073,12.034979,16.5,296.9,16.5,30.546437,18.51183,21.635442,37.560492,18.965517,7.064364,37286,69.341408,3200,19.843750,21.937500,44.593750,0.468750,1.093750,0.062500,5.687500,47.312500,4,9457.189341,18.512541,3.903933,3.786131,13.551402,18.219631,32.5,5.0,27.4,5.824399,13.818968,60.000000,408.8,36.474164,16.065054,47.606706,NaN,0.00000,3200:0,31.250000,3200:1,7961.0,55.0,22.0,90.000000,49.513213,6.311475,40.0,79571,12956,6.141633,320.081387,68.212824,10.8,No,11.627907,104,461.8,26,16421,110.5
4,5,13009,Georgia,Baldwin,75.244221,474.0,97.237316,9.987516,14.951144,14.317559,12.4,319.9,22.6,9.319845,18.51183,15.711363,38.618257,18.297424,5.704281,40358,55.137783,44906,19.175611,15.594798,42.199261,0.345165,1.719147,0.077941,2.280319,52.373848,5,9659.224156,23.503294,4.702728,4.250728,12.649628,24.107593,34.1,5.8,23.9,63.007437,13.976129,19.444444,635.7,17.809268,15.034754,57.593480,1736:1,40.08373,2495:1,133.612430,

In [227]:
#master_df.columns

## Dropping some reduntant columns

In [228]:
master_df = master_df.drop(['level_0_x', 'Age_Adjusted_Mortality', 'pct_Frequent_Physical_Distress', 'pct_Frequent_Mental_Distress',
       'pct_Adult_Uninsured', 'pct_Child_Uninsured','Population_x', 'PCP_Ratio',
       'Dentist_Ratio', 'MHP_Ratio', '80th_Percentile_Income',
       '20th_Percentile_Income', 'CaseCount', 'DeathCount', 'Population_y'
        ], axis=1)

# Exporting cleaned data frame to csv 04-26-2019

In [229]:
master_df.to_csv('master.csv', index = False)

In [230]:
master = pd.read_csv('./master.csv')
master.head()

,FIPS,State,County,Life_Expectancy,Child_Mortality_Rate,Infant_Mortality_Rate,pct_Diabetic,HIV_Prevalence_Rate,pct_Food_Insecure,pct_Limited_Access_to_healthy_foods,Drug_Overdose_Mortality_Rate,MV_Mortality_Rate,pct_Insufficient_Sleep,Household_Income,pct_Homeowners,pct_below_18,pct_65_and_over,pct_African_American,pct_American_Indian_Alaskan_Native,pct_Asian,pct_Native_Hawaiian_Other_Pacific_Islander,pct_Hispanic,pct_Non_Hispanic_White,level_0_y,Years_of_Potential_Life_Lost_Rate,pct_Fair_Poor,Physically_Unhealthy_Days,Mentally_Unhealthy_Days,pct_LBW,pct_Smokers,pct_Obese,Food_Environment_Index,pct_Physically_Inactive,pct_With_Access,pct_Excessive_Drinking,pct_Alcohol_Impaired,Chlamydia_Rate,Teen_Birth_Rate,pct_Uninsured,PCP_Rate,Dentist_Rate,MHP_Rate,Preventable_Hosp._Rate,pct_Screened,pct_Vaccinated,Graduation_Rate,pct_Some_College,pct_Unemployed,pct_Children_in_Poverty,Income_Ratio,Violent_Crime_Rate,Injury_Death_Rate,Average_Daily_PM2.5,Presence_of_violation,pct_Severe_Housing_Problems,Cases_AgeAdj_Rate,Deaths_AgeAdj_Rate
0,13001,Georgia,Appling,74.553535,59.436970,7.636473,12.9,172.0,14.3,6.773041,18.51183,23.225027,36.040842,40862,71.859147,24.793478,17.266886,18.595108,0.577723,0.788294,0.205173,10.258625,69.051347,1,10749.968790,20.860321,4.315623,4.167643,9.866512,20.200401,36.1,7.5,27.2,27.648607,14.686055,24.000000,536.5,66.978193,17.608928,43.412200,16.19783,37.794940,7080.0,37.0,30.0,82.683983,38.746241,5.528561,33.5,5.098134,379.198267,80.101317,10.6,No,10.205580,402.5,179.5
1,13003,Georgia,Atkinson,73.462140,69.199726,7.636473,12.8,240.8,12.8,12.267063,18.51183,36.068220,38.392034,34821,74.074074,27.307600,14.241189,16.099257,1.498442,0.911053,1.066890,24.814193,56.808919,2,10902.115783,26.766415,4.863178,4.439941,7.602957,23.552669,32.1,7.3,28.2,0.000000,14.423521,37.500000,428.7,78.017664,24.172885,47.606706,11.98753,105.250941,7663.0,35.0,36.0,87.037037,35.566038,4.495192,36.5,4.226623,320.081387,105.957713,10.4,No,15.884477,427.4,203.0
2,13005,Georgia,Bacon,72.577063,69.199726,7.636473,12.4,468.6,13.2,5.476352,18.51183,31.690498,36.434163,38417,70.719479,25.894514,15.434226,16.043820,0.353388,0.600760,0.167859,8.905380,72.877463,3,11414.503045,19.157643,4.331201,4.020762,9.456740,19.999158,33.0,7.8,28.9,61.094088,15.757069,23.529412,522.2,47.545419,18.625040,43.967640,26.50411,8.834700,7038.0,36.0,32.0,84.671533,42.101740,4.537419,42.7,5.370769,320.081387,88.515942,10.4,No,10.126582,452.9,215.3
3,13007,Georgia,Baker,76.226226,69.199726,7.636473,16.5,296.9,16.5,30.546437,18.51183,21.635442,37.560492,37286,69.341408,19.843750,21.937500,44.593750,0.468750,1.093750,0.062500,5.687500,47.312500,4,9457.189341,18.512541,3.903933,3.786131,13.551402,18.219631,32.5,5.0,27.4,5.824399,13.818968,60.000000,408.8,36.474164,16.065054,47.606706,0.00000,31.250000,7961.0,55.0,22.0,90.000000,49.513213,6.311475,40.0,6.141633,320.081387,68.212824,10.8,No,11.627907,461.8,110.5
4,13009,Georgia,Baldwin,75.244221,97.237316,9.987516,12.4,319.9,22.6,9.319845,18.51183,15.711363,38.618257,40358,55.137783,19.175611,15.594798,42.199261,0.345165,1.719147,0.077941,2.280319,52.373848,5,9659.224156,23.503294,4.702728,4.250728,12.649628,24.107593,34.1,5.8,23.9,63.007437,13.976129,19.444444,635.7,17.809268,15.034754,57.593480,40.08373,133.612430,5451.0,40.0,40.0,88.813559,52.376673,5.944429,32.5,6.157213,689.414434,72.101540,11.4,Yes,20.025031,479.0,188.5
